Install libraries

In [ ]:
!pip install gensim --upgrade -q

     |████████████████████████████████| 24.1 MB 1.9 MB/s 


In [ ]:
import gensim
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
import tqdm
import nltk
import re
import plotly.graph_objects as go

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Clean dataset

In [ ]:
def preprocess_text(sentence):
    # Lowercase
    sentence = sentence.lower()
    
    # Remove all non-alphabets (punctuation, numbers, new-line characters and extra-spaces)
    sentence = re.sub(r'[^a-zA-Z]+', ' ', sentence)
    sentence = sentence.replace('\n', '')
    # sentence = re.sub('\s\s+', ' ', sentence)
    
    # Tokenize & remove stop-words
    word_list = nltk.word_tokenize(sentence)    
    stopwords_list = set(nltk.corpus.stopwords.words('english'))
    word_list = [word for word in word_list if word not in stopwords_list]
    
    # Remove very small words, length < 3, they don't contribute any useful information
    word_list = [word for word in word_list if len(word) > 3]
        
    # Stem & Lemmatize
    porter_stemmer = nltk.stem.PorterStemmer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    word_list = [porter_stemmer.stem(word) for word in word_list]
    word_list = [lemmatizer.lemmatize(word) for word in word_list]
    
    sentence = ' '.join(word_list)
    
    return sentence

Load dataset

In [ ]:
from sklearn.datasets import fetch_20newsgroups
news_group = fetch_20newsgroups(subset='train')

news_group_data = news_group.data
news_group_target_names = news_group.target_names
news_group_target = news_group.target

# Creating a dataframe from the loaded data
news_df = pd.DataFrame({'news': news_group_data, 
                        'class': news_group_target})

news_df.reset_index(drop=True, inplace= True)

from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(news_df, test_size=0.2)

Pre-process dataset

In [ ]:
# Preprocess the news description
tqdm.tqdm.pandas()
train_df['news_tokenized'] = train_df['news'].progress_apply(lambda x: preprocess_text(str(x)))


100%|██████████| 9051/9051 [01:12<00:00, 124.60it/s]


TF-IDF Vectors

In [ ]:
tf_idfvectorizer = TfidfVectorizer(strip_accents='unicode', stop_words='english', min_df=2)
x_tfidf = tf_idfvectorizer.fit_transform(train_df.news_tokenized)

Create dataframe

In [ ]:
performance_metrics = pd.DataFrame(columns=['feature-extraction','run#', 'epoch', 'state', 'AMI','ARI','NMI'])


NMF Epochs Grid search

In [12]:

for run in range(1, 3, 1):
    print('Run #', run)
    for epoch, state in zip(range(25, 250, 50), range(2, 42, 2)):
        k_means = KMeans(n_clusters=20, init='k-means++', max_iter=epoch, random_state=state)
        k_means.fit(x_tfidf)
        
        ami = metrics.adjusted_mutual_info_score(train_df['class'], k_means.labels_)
        ari = metrics.adjusted_rand_score(train_df['class'], k_means.labels_)
        nmi = metrics.normalized_mutual_info_score(train_df['class'], k_means.labels_)

        print(run, epoch, state)
        
        performance_metrics = performance_metrics.append({'feature-extraction':'tf-idf', 'run#':run, 'epoch':epoch, 'state':state, 'AMI':ami, 'ARI': ari, 'NMI':nmi}, ignore_index=True)


Run # 1
1 25 2
1 75 4
1 125 6
1 175 8
1 225 10
Run # 2
2 25 2
2 75 4
2 125 6
2 175 8
2 225 10


Save metrics

In [14]:
performance_metrics.to_csv('/content/drive/MyDrive/Big Data/Code/Metrics/performance-tfidf_kmeans.csv', index=False)
performance_metrics[['run#','epoch','state','AMI','ARI','NMI']] = performance_metrics[['run#','epoch','state','AMI','ARI','NMI']].apply(pd.to_numeric, errors='coerce')
performance_metrics.dtypes
mean_performance = performance_metrics.groupby('epoch', as_index=False)[['AMI','ARI','NMI']].mean()
mean_performance.to_csv('/content/drive/MyDrive/Big Data/Code/Metrics/mean_performance-tfidf_kmeans.csv', index=False)